In [ ]:
# Suggested data science package imports
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt

# TrendMiner package import
import trendminer
from trendminer.trendminer_client import TrendMinerClient

import os

token = os.environ["KERNEL_USER_TOKEN"]
serverUrl = os.environ["KERNEL_SERVER_URL"]

# Create TrendMiner API object
client = TrendMinerClient(token)

In [ ]:
import pytz
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import datetime as dt
import json
import matplotlib.ticker as ticker
import requests
import calendar

auth_header = {'Authorization' : f'Bearer {token}'}

In [ ]:
def generate_date_ranges(start, end, frequency, target_timezone):
    
    if frequency == 'daily':
        dates = get_day_range(start_date, end_date, target_timezone)
    elif frequency == 'monthly':
        dates = get_month_range(start_date, end_date, target_timezone)
    else:
        dates = get_week_range(start_date, end_date, target_timezone)

    return dates


def get_month_range(start_date, end_date, target_timezone):
    
    timezone = pytz.timezone(target_timezone)
    months = []
    current_month = datetime(start_date.year, start_date.month, 1)
    end_date = end_date
    
    while current_month < end_date:
        next_month = current_month.replace(day=1) + timedelta(days=32)
        start_of_next_month = next_month.replace(day=1)
        
        month_dict = {
            'startDate': current_month.replace(hour=int(timezone.utcoffset(current_month).total_seconds() / 3600 * -1), minute=0, second=0),
            'endDate': start_of_next_month.replace(hour=int(timezone.utcoffset(start_of_next_month).total_seconds() / 3600 * -1), minute=0, second=0),
            'key': f'key{len(months) + 1}'
        }
        
        months.append(month_dict)
        current_month = start_of_next_month
    
    dates = pd.DataFrame(months)
    
    dates['startDate'] = dates['startDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    dates['endDate'] = dates['endDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    return dates


def get_day_range(start_date, end_date, target_timezone):
    timezone = pytz.timezone(target_timezone)
    days = []
    current_day = start_date
    end_date = end_date # Ensure end_date has timezone information
    
    
    while current_day < end_date:
        start_of_next_day = current_day + timedelta(days=1)
                
        days_dict = {
            'startDate': current_day.replace(hour=int(timezone.utcoffset(current_day).total_seconds() / 3600 * -1), minute=0, second=0),
            'endDate': start_of_next_day.replace(hour=int(timezone.utcoffset(start_of_next_day).total_seconds() / 3600 * -1), minute=0, second=0),
            'key': f'key{len(days) + 1}'
        }
        
        days.append(days_dict)
        current_day = start_of_next_day
    
    dates = pd.DataFrame(days)
    
    dates['startDate'] = dates['startDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    dates['endDate'] = dates['endDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    return dates


def get_week_range(start_date, end_date, target_timezone):
    timezone = pytz.timezone(target_timezone)
    current_date = start_date
    end_date = end_date  # Localize end_date
    
    weeks = []
    
    while current_date <= end_date:
        current_week = current_date - timedelta(days=current_date.weekday())
        start_of_next_week = current_week + timedelta(days=7)
        
        week_dict = {
            'startDate': current_week.replace(hour=int(timezone.utcoffset(current_week).lo() / 3600 * -1), minute=0, second=0),
            'endDate': start_of_next_week.replace(hour=int(timezone.utcoffset(start_of_next_week).total_seconds() / 3600 * -1), minute=0, second=0),
            'key': f'key{len(weeks) + 1}'
        }
        
        weeks.append(week_dict)
        current_date = start_of_next_week
        
    dates = pd.DataFrame(weeks)
    
    dates['startDate'] = dates['startDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    dates['endDate'] = dates['endDate'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    return dates


def get_calculations(tagname, interpolation, agg, dateRanges):
    
    request_body = {
        "queries": {
        "tag": {
            "id": tagname,
            "shift": 0,
            "interpolationType": interpolation
        },
        "timePeriods": dateRanges
        },
        "parameters": {
        "calculationType": agg
        }
    }
    
    r = requests.post(f'{serverUrl}/compute/newCalculation', json=request_body, headers=auth_header)

    return r.json()


def get_tag_details(tagname):
    
    params = {'tagName': tagname}
    
    r = requests.get(f'{serverUrl}/hps/api/tags/details', params=params, headers=auth_header)
    
    return r.json()



def get_tags_timebased_calculations(tags_aggs, tags_compound_calculations, timePeriods):
    
    results = pd.DataFrame()

    for calcName, tag_agg in tags_aggs.items():
        try:
            tag_details = get_tag_details(tag_agg[0])
            interpolationType = tag_details['interpolationType']

            # GETTING CALCULATIONS FROM TRENDHUB 
            calculations = get_calculations(tag_agg[0], interpolationType, tag_agg[1], timePeriods)

            df = pd.DataFrame(calculations)
            df = df.rename(columns={'value': calcName})

            if results.empty:
                results = df
            else:
                results = pd.concat([results, df[calcName]], axis=1)
        except:
            pass
        
    for tags_compound_calculation in tags_compound_calculations:
        try:
            results[tags_compound_calculation] = eval(tags_compound_calculations[tags_compound_calculation].replace("[", "results["))
        except:
            print("**** Error Performing Calculation: ", tags_compound_calculations[tags_compound_calculation])
            pass
    
    
    results[['startDate', 'endDate']] = results[['startDate', 'endDate']].apply(pd.to_datetime)
    
    results = results.drop('key', axis=1)
    
    results = results.sort_values('startDate', ascending = True)

    return results

In [ ]:
############################ CONFIGURATION ##############################

target_timezone = 'US/Central'

start_date = datetime(datetime.now().year, 1, 1)       # datetime(2023, 5, 27)
end_date = datetime(datetime.now().year + 1, 1, 1)

frequency = 'monthly'   # Options: daily, monthly, weekly

tags_aggs = {     # Define the tags and aggregations to retrieve data for:
    "SomeName": ("BA:LEVEL.1", "integral"),
    "SomeName2": ("BA:CONC.1", "mean"),
    "SomeName3": ("BA:TEMP.1", "stdev")
}

tags_compound_calculations = {
    "someCalculationName1": "['SomeName'] - ['SomeName2']"
}

############################ EXECUTION ##############################

dates = generate_date_ranges(start_date, end_date, frequency, target_timezone)

myCalculations = get_tags_timebased_calculations(tags_aggs, tags_compound_calculations, dates.to_dict('records'))

myCalculations